<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/w2v2_finetune_on_asr_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch torchaudio
!pip install transformers[torch]

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from datasets import load_dataset
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Trainer

# Load the dataset
dataset = load_dataset("HamdanXI/uclass_asr_v3")

def prepare_dataset(batch):
    # Process audio
    speech_array = batch['audio']['array']
    speech_tensor = torch.tensor(speech_array)

    # Pad to the maximum length
    max_length = 205 * 16000  # 205 seconds at 16kHz
    padded_speech = torch.nn.functional.pad(
        input=speech_tensor,
        pad=(0, max_length - speech_tensor.shape[0]) if speech_tensor.shape[0] < max_length else (0, 0),
        mode='constant',
        value=0.0
    )

    # Tokenize the labels
    with processor.as_target_processor():
        labels = processor(batch['transcript']).input_ids

    return {"input_values": padded_speech.numpy(), "labels": labels}

# Load tokenizer and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# Apply the mapping function
dataset = dataset.map(prepare_dataset, remove_columns=["audio"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    group_by_length=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    save_total_limit=2
)

import torch
from torch.nn.utils.rnn import pad_sequence

class DataCollatorForWav2Vec2:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # Ensure each input value is a tensor, then pad
        input_values = [torch.tensor(feature['input_values'], dtype=torch.float) for feature in features]
        labels = [torch.tensor(feature['labels'], dtype=torch.long) for feature in features]

        # Pad input values and labels
        input_values_padded = pad_sequence(input_values, batch_first=True, padding_value=0.0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=self.processor.tokenizer.pad_token_id)

        return {
            'input_values': input_values_padded,
            'labels': labels_padded
        }

data_collator = DataCollatorForWav2Vec2(processor=processor)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
    tokenizer=processor
)

# Testing the trainer on a small subset to ensure it processes data correctly
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you ar